In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

sk
Fy
tvly


In [11]:
import warnings
warnings.filterwarnings("ignore")

from langchain_community.vectorstores import FAISS
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_core.tools import tool
from langchain.agents import tool
from langchain_community.tools import TavilySearchResults

from langchain_openai import ChatOpenAI
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

# LangGraph MessagesState라는 미리 만들어진 상태를 사용
from langgraph.graph import MessagesState
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import create_react_agent

from textwrap import dedent
from typing import List, Literal, Tuple
from pydantic import BaseModel, Field


from pprint import pprint
import re

import uuid
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

def extract_menu_info(doc: Document) -> dict:
    """Vector DB 문서에서 구조화된 메뉴 정보 추출"""
    content = doc.page_content
    # Document의 metadata에 메뉴명이 있는 경우 사용, 없으면 내용에서 추출 시도
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    # 정규표현식으로 가격, 설명 등 추출
    price_match = re.search(r'₩([\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    
    # metadata에 메뉴명이 없을 경우 content에서 "메뉴:" 부분을 찾아서 사용
    if menu_name == 'Unknown':
        name_match = re.search(r'메뉴:\s*(.+?)(?:,|$)', content)
        menu_name = name_match.group(1).strip() if name_match else "메뉴명 불명"

    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

@tool
def db_search_cafe_func(query: str) -> str:
    """
    특정 카페의 메뉴 정보, 가격, 또는 관련 세부 정보를 찾을 때 사용합니다. 
    검색된 문서에서 핵심 정보(이름, 가격, 설명)를 추출하여 요약된 JSON 배열 형태로 반환합니다.
    """
    docs = cafe_db.similarity_search(query, k=4)
    
    if len(docs) == 0:
        return "관련 카페 메뉴 정보를 찾을 수 없습니다."
    
    # extract_menu_info를 사용하여 구조화된 정보로 변환
    structured_info = [extract_menu_info(doc) for doc in docs]
    
    # LLM에게 전달할 텍스트 형태로 변환 (JSON 문자열이나 가독성 좋은 텍스트)
    context_text = "검색된 메뉴 정보:\n"
    for item in structured_info:
        context_text += f"- 메뉴명: {item['name']}, 가격: {item['price']}, 설명: {item['description'][:50]}...\n"
        
    return context_text
# LLM 모델 
#llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
)

# 도구 목록
tools = [db_search_cafe_func]
system_prompt = dedent("""
    당신은 친절한 카페 메뉴 안내 AI입니다.
    사용 가능한 도구 'db_search_cafe_func'를 사용하여 질문에 필요한 정보를 검색하고,
    검색된 **구조화된 메뉴 정보**를 바탕으로 한국어로 친절하고 정확하게 답변하세요.
""")

llm_with_tools=llm.bind_tools(tools)
class GraphState(MessagesState):
    pass

def call_model(state: GraphState):
    # 시스템 메시지를 정의하여 LLM의 페르소나와 역할을 설정합니다.
    system_message = SystemMessage(content=system_prompt)
    # 기존 메시지 목록 앞에 시스템 메시지를 추가합니다.
    messages = [system_message] + state['messages']
    # 도구 호출 기능이 활성화된 LLM을 호출하여 응답을 받습니다.
    response = llm_with_tools.invoke(messages)
    # LLM의 응답을 상태에 저장하여 반환합니다.
    return {"messages": [response]}

def should_continue(state: GraphState):
    # 가장 마지막 메시지를 가져옵니다.
    last_message = state["messages"][-1]
    
    # 마지막 메시지에 도구 호출이 포함되어 있으면, "execute_tools" 노드로 이동합니다.
    if last_message.tool_calls:
        return "execute_tools"
    # 도구 호출이 없으면, 대화를 종료합니다.
    return END


builder = StateGraph(GraphState)
builder.add_node("call_model", call_model)
builder.add_node("execute_tools", ToolNode(tools))

builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    should_continue,
    {
        # 'should_continue'가 "execute_tools"를 반환하면, 해당 노드로 이동합니다.
        "execute_tools": "execute_tools",
        # 'should_continue'가 END를 반환하면, 그래프를 종료합니다.
        END: END
    }
)
builder.add_edge("execute_tools", "call_model")
my_graph = builder.compile()



In [12]:

query = "카페라떼는 있나요?"
messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=query)
    ]
inputs = {"messages": messages}
# my_graph 변수는 직접 StateGraph를 생성하고 노드와 엣지를 추가
messages = my_graph.invoke(inputs)

for m in messages['messages']:
    m.pretty_print()

================================ System Message ================================


당신은 친절한 카페 메뉴 안내 AI입니다.
사용 가능한 도구 'db_search_cafe_func'를 사용하여 질문에 필요한 정보를 검색하고,
검색된 **구조화된 메뉴 정보**를 바탕으로 한국어로 친절하고 정확하게 답변하세요.

================================ Human Message =================================

카페라떼는 있나요?
================================== Ai Message ==================================

[해당 카페에 "카페라떼" 메뉴가 존재하는지 확인하기 위해 필수적인 검색 함수입니다. 메뉴 유무 및 가격/설명 등 핵심 정보를 추출하여 정확히 안내하기 위해 필요합니다.]  

검색 결과를 확인한 후 다음과 같이 답변드리겠습니다:  
"예, 카페라떼는 현재 [가격]원에 판매 중이며, [설명]입니다. 주문 시 참고해 주세요!"  
(또는 "죄송합니다. 현재 카페라떼는 메뉴에 없는 것으로 확인됩니다.")
Tool Calls:
  db_search_cafe_func (chatcmpl-tool-88a0fae13b9245e588f3463a2d06ae8b)
 Call ID: chatcmpl-tool-88a0fae13b9245e588f3463a2d06ae8b
  Args:
    query: 카페라떼
================================= Tool Message =================================
Name: db_search_cafe_func

검색된 메뉴 정보:
- 메뉴명: 카페라떼, 가격: ₩5,500, 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질...
- 메뉴명: 바닐라 라떼, 가격: ₩6,